In [180]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST

In [181]:
train_dataset = MNIST(root='./data', train=True, download=True)

In [182]:
test_dataset = MNIST(root='./data', train=False, download=True)

In [183]:
# Для обучающего набора преобразуем массив в np вид и нормализуем, чтобы оттенок пикселя был дробным числом от 0 до 1
train_images_tensor = train_dataset.data.clone().detach() / 255
train_images_tensor[0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [184]:
test_images_tensor = test_dataset.data.clone().detach() / 255
train_labels_tensor = train_dataset.targets.clone().detach()
test_labels_tensor = test_dataset.targets.clone().detach()

In [185]:
num_images = len(train_labels_tensor)
print("Количество изображений в train_dataset:", num_images)

Количество изображений в train_dataset: 60000


In [186]:
# определение архитектуры нейронной сети
# nn.Module - это базовый класс в библиотеке PyTorch, который используется для определения нейронных сетей
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten() # ф-ция преобразование изображения в вектор
        self.fc1 = nn.Linear(28 * 28, 128) # полносвязанный слой с 128 нейронами
        self.fc2 = nn.Linear(128, 10) # выходной слой с 10 нейронами (для 10 цифр)

    def forward(self, x):
        x = self.flatten(x) # разворачиваем в один вектор
        x = torch.relu(self.fc1(x)) # Применение ReLU к выходу первого слоя
        return self.fc2(x) # Применение второго полносвязанного слоя

In [197]:
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1)

In [201]:
num_epochs = 300
for epoch in range(num_epochs):
    optimizer.zero_grad() # Обнуление градиентов
    outputs = model(train_images_tensor) # Получение выхода модели
    loss = criterion(outputs, train_labels_tensor) # Вычисление потерь
    loss.backward() # Обратное распространение ошибки
    optimizer.step() # Обновление весов

print(f'Loss: {loss.item():.4f}')

Loss: 0.1117


In [202]:
# оценка производительности модели на тестовом наборе данных
model.eval()
correct = 0
total = len(test_labels_tensor)
with torch.no_grad():
    outputs = model(test_images_tensor)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == test_labels_tensor).sum().item()

print(f'Точность при проверке на тестовом наборе: {100 * correct / total:.2f}%')

Точность при проверке на тестовом наборе: 96.47%
